# This Notebook
# ... Is Super Junky. As long as this note is here, maybe assume
# you can't run it.

In here, everything on which we're planning to train gets merged together and dumped to a CSV and a pickle for quick reference.

This pickle is **keyed on dd_id**. But it's **phone numbers** that are relevant for the classifier.


------

# Setup
## Imports

In [1]:
import ujson as json
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from itertools import chain
from tqdm import tqdm, tqdm_pandas
from sqlalchemy import create_engine
%matplotlib inline

from helpers import phone_str_to_dd_format
from helpers import disaggregated_df
from helpers import aggregated_df
from helpers import dummify_df

In [2]:
def df_of_tables_for_dd_ids(dd_ids, sqlite_tables, sql_con):
    """
    :param list dd_ids: list of Deep Dive IDs to retrieve
    :param list sqlite_tables: list of SQLite tables to join
    :param sqlalchemy.create_engine sql_con: Connection to SQLite (can be \
    omitted)
    :returns: `pandas.DataFrame` -- dataframe of tables, joined using the Deep \
    Dive IDs.
    """
    import pandas as pd

    dd_ids_str = ','.join(['"{}"'.format(x) for x in dd_ids])
    query_fmt = 'select * from {} where dd_id in ({})'.format

    df = pd.read_sql(query_fmt(sqlite_tables[0], dd_ids_str), sql_con).drop_duplicates()
    df['dd_id'] = df.dd_id.astype(int)

    for s_t in sqlite_tables[1:]:
        df_2 = pd.read_sql(query_fmt(s_t, dd_ids_str), sql_con)
        df_2['dd_id'] = df_2.dd_id.astype(int)
        
        # We use outer joins because dd_ids in one table may be missing from the other.
        df = df.merge(df_2, on=['dd_id'], how='outer')

    if 'post_date' in df:
        df['post_date'] = df.post_date.apply(pd.to_datetime)
        
    if 'duration_in_mins' in df:
        df['duration_in_mins'] = df.duration_in_mins.apply(lambda x: float(x) if x != '' else np.nan)
        
    # I melted some rows when making this, and it's proven a mistake. Let's unmelt
    melted_cols = ['ethnicity', 'flag']
    for m_c in melted_cols:
        if m_c in df.columns:
            df = aggregated_df(df, m_c, 'dd_id', '|')

    return df

# Read Training Data

In [3]:
jsns = [json.loads(x) for x in open('../../data/orig_ht_data/ht_training.json', 'r')]
train_df = pd.DataFrame.from_records(jsns)
train_df = train_df.ix[:, ['class', 'phone', 'url']]

In [4]:
print(train_df.shape)
train_df['phone'] = train_df.phone.apply(lambda x:','.join(x))
train_df = disaggregated_df(train_df, 'phone', ',')
print(train_df.shape)
train_df = train_df.ix[train_df.phone != 'nan', :]
print(train_df.shape)

train_df['phone'] = train_df.phone.apply(phone_str_to_dd_format)

(4796, 3)
(4192, 3)
(4169, 3)


We drop phone numbers that have both positive and negative values. This is an oversimplification, and we could also try something like calculating a real score. This is okay for now.

In [5]:
pos_neg_set = \
set(train_df.ix[(train_df['class'] == 'positive'), 'phone']) &\
set(train_df.ix[(train_df['class'] == 'negative'), 'phone'])
train_df = train_df.ix[~train_df.phone.isin(pos_neg_set), :]
print(train_df.shape)

(4034, 3)


We're not using urls, so we drop them too. We can put these back if we want them.

In [6]:
train_df = train_df.ix[:, ['class', 'phone']].drop_duplicates()
print(train_df.shape)

(67, 2)


## Read in random ad phone numbers
Selected by Feng! 2 million, exclude those in the training data, exclude those with over a thousand associated ads.
We're taking these as **negative** examples.

In [7]:
null_phones = open('../../data/lattice/cp1_training_with_random/training_phones_random.txt', 'r').readlines()
null_phones = [x.strip() for x in null_phones]

In [8]:
train_df = pd.concat([train_df, pd.DataFrame({'class': ['negative']*len(null_phones), 'phone': null_phones})])
print(train_df.shape)

(567, 2)


# Read DeepDive Data w/ Training Phones

In [9]:
sql_con = create_engine('sqlite:////Users/pmlandwehr/wkbnch/memex/memex_queries/dd_dump.db')
query_str_fmt = 'select {} from {} where {} in ({})'.format

In [10]:
phone_str_list = ','.join(['"{}"'.format(x) for x in train_df.phone.unique()])
query_str = query_str_fmt('*', 'dd_id_to_phone', 'phone', phone_str_list)
df = pd.read_sql(query_str, sql_con)
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(53005, 2)
(29027, 2)


In [11]:
df_2 = df_of_tables_for_dd_ids(list(df.dd_id.unique()),
                                ['dd_id_to_price_duration',
                                'dd_id_to_flag',
                                'dd_id_to_age',
                                'dd_id_to_cbsa',
                                'dd_id_to_ethnicity',
                                'dd_id_to_cdr_id'],
                               sql_con)
print(df_2.shape)

(28512, 9)


For those ads with prices and durations, let's add a price per minute value

In [12]:
df_2['price_per_min'] = df_2.price / df_2.duration_in_mins

In [13]:
## DD IDs in the first list are definitive and complete, so left join
df_3 = df.merge(df_2, on='dd_id', how='left')
print(df_3.shape)

(29519, 11)


**Clean up**

In [14]:
del df
del df_2

## Join Deep Dive Data with Training Data
This **very vanilla** join is sort of insufficient because it keepts eveything keyed on `dd_ids`. It should really be keyed on unique phone numbers, with ad-level stats getting aggregated.

In [15]:
df_4 = train_df.merge(df_3, on='phone', how='left')
print(df_4.shape)

(29531, 12)


**Clean up**

In [16]:
del df_3

# Join Deep Dive Data with Greg's HT Data
## STD Data
It looks like MSA, CBSA, and Name all match to the same values, so could drop some colums

In [17]:
std_df = pd.read_excel('../../data/greg_correlates/std.xlsx')

In [18]:
std_df.head()

,MSA,CBSA,Name,Disease,Year,Cases,Rate
0,31000US12060,12060,"Atlanta-Sandy Springs-Roswell, GA",Chlamydia,2009,20337,370.2
1,31000US12420,12420,"Austin-Round Rock, TX",Chlamydia,2009,8456,495.9
2,31000US12580,12580,"Baltimore-Columbia-Towson, MD",Chlamydia,2009,12883,478.8
3,31000US13820,13820,"Birmingham-Hoover, AL",Chlamydia,2009,6120,541.1
4,31000US14460,14460,"Boston-Cambridge-Newton, MA-NH",Chlamydia,2009,13285,289.5


In [19]:
df_4 = df_4.merge(std_df.ix[:, ['Name', 'Disease', 'Year', 'Cases', 'Rate', 'MSA']],
                  left_on='area',
                  right_on='Name',
                  how='left')
del df_4['Name']
print(df_4.shape)

(144121, 17)


**Clean up**

In [20]:
del std_df

## MSA Characteristics
Note that we're using the **yearly** version of the file. We could also use the **monthly** version. I'm primarily choosing yearly because monthly had some import issues that it doesn't seem worth wrangling at this second.

In [21]:
msa_df = pd.read_csv('../../data/greg_correlates/msa_characteristics.csv')

In [22]:
msa_df.head()

,census_msa_code,property,rape,violent,male_wage_mean,male_wage_p05,male_wage_p10,male_wage_p25,male_wage_p50,male_wage_p75,...,wage_sum.wght,female_epop,swnauthemp,swnftemp,ad_p10_msa,ad_p90_msa,ad_mean_msa,ad_median_msa,ad_count_msa,ad_p50_msa
0,31000US10180,5538.60,63.00,593.8,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,197.0,179.0,80.000000,275.219812,169.981313,150.000000,1148.0,150.000000
1,31000US10420,21815.75,273.75,2182.0,30.566309,6.35,9.30,14.8,22.50,35.5,...,363283.0,0.463908,1034.0,975.0,83.333333,206.404629,134.607090,115.000000,4503.0,115.000000
2,31000US10500,6969.60,40.60,961.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,80.000000,251.629542,150.384258,140.000000,1565.0,140.000000
3,31000US10580,21216.20,175.40,2430.0,31.147955,7.00,9.75,15.0,25.00,37.5,...,434363.0,0.472414,NaN,NaN,90.000000,252.108219,165.443290,157.268464,3711.0,157.268464
4,31000US10740,37465.25,453.25,6314.5,26.628241,5.70,7.50,12.0,20.25,34.0,...,451024.0,0.400557,1347.0,1208.0,87.500000,244.352807,156.882813,144.974193,9978.0,144.974193


In [23]:
df_4 = df_4.merge(msa_df,
                  left_on='MSA',
                  right_on='census_msa_code',
                 how='left')
del df_4['census_msa_code']
print(df_4.shape)

(144121, 60)


**Clean up**

In [24]:
del msa_df

# Save the Results

In [25]:
df_4.to_csv('../../data/merged/data_to_use_by_ad.csv', index=False)
df_4.to_pickle('../../data/merged/data_to_use_by_ad.pkl')